In [18]:
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, Adadelta, Adagrad
from keras.utils import np_utils, generic_utils
from keras.callbacks import EarlyStopping
from keras.layers.advanced_activations import PReLU, LeakyReLU
from keras.layers import Embedding,GRU,TimeDistributed,RepeatVector, LSTM, concatenate , Input, Reshape
from keras.preprocessing.text import one_hot
from keras.preprocessing import sequence
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import numpy as np
from keras import backend as K 
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.preprocessing import image
from keras.applications.inception_resnet_v2 import preprocess_input
from keras.models import Model
K.set_image_dim_ordering('th')

In [19]:
max_caption_len = 4
vocab_size = 4

In [20]:
img_path = 'screenshot.jpg'
img = image.load_img(img_path, target_size=(299, 299))
img = image.img_to_array(img)
images = []
for i in range(3):
    images.append(img)
images = np.expand_dims(images, axis=0)
images = preprocess_input(images)
images = images.reshape(3, 3, 299, 299)

In [21]:
html_input = to_categorical([[0, 0, 0, 1], [0, 0, 1, 2], [0, 1, 2, 3]])
html_input = html_input.reshape(3,4,4)
next_words = to_categorical([[1, 2, 3]])

In [22]:
IR2 = InceptionResNetV2(weights=None, include_top=True)
ir2_out = Dense(1024, activation='relu')(IR2.output)
ir2_out = RepeatVector(max_caption_len)(ir2_out)
print(ir2_out.shape)

(?, 4, 1024)


In [23]:
language_input = Input(shape=(vocab_size, vocab_size))
language_model = LSTM(128, return_sequences=True)(language_input)
print(language_model.shape)

(?, ?, 128)


In [24]:
print(ir2_out.shape, language_model.shape)
decoder = concatenate([ir2_out, language_model])
decoder = LSTM(512, return_sequences=False)(decoder)
decoder_output = Dense(vocab_size, activation='softmax')(decoder)
# model.summary()
print(decoder.shape)
model = Model(inputs=[IR2.input, language_input], outputs=decoder_output)
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
model.summary()

(?, 4, 1024) (?, ?, 128)
(?, 512)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 3, None, None 0                                            
__________________________________________________________________________________________________
conv2d_407 (Conv2D)             (None, 32, None, Non 864         input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_407 (BatchN (None, 32, None, Non 96          conv2d_407[0][0]                 
__________________________________________________________________________________________________
activation_407 (Activation)     (None, 32, None, Non 0           batch_normalization_407[0][0]    
___________________________________________________________________________

batch_normalization_446 (BatchN (None, 32, None, Non 96          conv2d_446[0][0]                 
__________________________________________________________________________________________________
activation_446 (Activation)     (None, 32, None, Non 0           batch_normalization_446[0][0]    
__________________________________________________________________________________________________
conv2d_444 (Conv2D)             (None, 32, None, Non 10240       block35_4_ac[0][0]               
__________________________________________________________________________________________________
conv2d_447 (Conv2D)             (None, 48, None, Non 13824       activation_446[0][0]             
__________________________________________________________________________________________________
batch_normalization_444 (BatchN (None, 32, None, Non 96          conv2d_444[0][0]                 
__________________________________________________________________________________________________
batch_norm

In [25]:
model.fit([images, html_input], next_words, batch_size=3, shuffle=False, epochs=5)

Epoch 1/5
3/3 [==============================] - 82s 27s/step - loss: 1.3900
Epoch 2/5
3/3 [==============================] - 17s 6s/step - loss: 1.2796
Epoch 3/5
3/3 [==============================] - 17s 6s/step - loss: 1.1021
Epoch 4/5
3/3 [==============================] - 16s 5s/step - loss: 0.9997
Epoch 5/5
3/3 [==============================] - 18s 6s/step - loss: 1.6324
